<a href="https://colab.research.google.com/github/rogerjordan123/html/blob/master/Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
!pip install python-mnist
from mnist import MNIST

In [41]:
mndata = MNIST('/content')

X_train, y_train = mndata.load_training()
X_test, y_test = mndata.load_testing()

X_train, y_train, X_test, y_test = np.array(X_train), np.array(y_train), np.array(X_test), np.array(y_test)


In [15]:
# MNIST data loader
#
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import struct
from array import array
from os.path  import join

def to_categorical(a, num_classes):
    return np.squeeze(np.eye(num_classes)[a.reshape(-1)])   

#
# MNIST Data Loader Class
#
class MnistDataloader(object):
    def __init__(self, training_images_filepath,training_labels_filepath,
                 test_images_filepath, test_labels_filepath):
        self.training_images_filepath = training_images_filepath
        self.training_labels_filepath = training_labels_filepath
        self.test_images_filepath = test_images_filepath
        self.test_labels_filepath = test_labels_filepath
    
    def read_images_labels(self, images_filepath, labels_filepath):        
        labels = []
        with open(labels_filepath, 'rb') as file:
            magic, size = struct.unpack(">II", file.read(8))
            if magic != 2049:
                raise ValueError('Magic number mismatch, expected 2049, got {}'.format(magic))
            labels = array("B", file.read())        
        
        with open(images_filepath, 'rb') as file:
            magic, size, rows, cols = struct.unpack(">IIII", file.read(16))
            if magic != 2051:
                raise ValueError('Magic number mismatch, expected 2051, got {}'.format(magic))
            image_data = array("B", file.read())        
        images = []
        for i in range(size):
            images.append([0] * rows * cols)
        for i in range(size):
            img = np.array(image_data[i * rows * cols:(i + 1) * rows * cols])
            img = img.reshape(28, 28)
            images[i][:] = img            
        
        return images, labels
            
    def load_data(self):
        print('Reading training data...')
        x_train, y_train = self.read_images_labels(self.training_images_filepath, self.training_labels_filepath)
        print('Reading test data...')
        x_test, y_test = self.read_images_labels(self.test_images_filepath, self.test_labels_filepath)
        print('MNIST data loaded.')
        return (x_train, y_train),(x_test, y_test)  

# Set file paths based on added MNIST Datasets
input_path = '../input'
training_images_filepath = join(input_path, '/content/train-images.idx3-ubyte')
training_labels_filepath = join(input_path, '/content/train-labels.idx1-ubyte')
test_images_filepath = join(input_path, '/content/train-images.idx3-ubyte')
test_labels_filepath = join(input_path, '/content/t10k-labels.idx1-ubyte')

# Load MINST dataset
mnist_dataloader = MnistDataloader(training_images_filepath, training_labels_filepath, test_images_filepath, test_labels_filepath)
(X_train, y_train), (X_test, y_test) = mnist_dataloader.load_data()

X_train = np.array(X_train)
y_train = np.array(y_train)

X_test = np.array(X_test)
y_test = np.array(y_test)

X_train = np.array([x.flatten() for x in X_train])
y_train = np.array([to_categorical(y, 10) for y in y_train], dtype=int)

X_test = np.array([x.flatten() for x in X_test])
y_test = np.array([to_categorical(y, 10) for y in y_test], dtype=int)

Reading training data...
Reading test data...
MNIST data loaded.


In [27]:
X_train.shape

(1000, 784)

In [55]:
X_train = X_train[0:5000,:]
X_test = X_test[0:500,:]
y_train = y_train[0:5000]
y_test = y_test[0:500]

In [56]:
class NeuralNetwork(object):

  def __init__(self,X,y):
    # m para os treinos de exemplos (quantas obsevações temos)
    self.m = X.shape[0]

    # n para numero de features no input data (32x32=784)
    self.n = X.shape[1]

    # h1 para o tamanho da sua primeira camada escondida
    self.h1 = 50

    # h2 para o tamanho da segunda camada escondida
    #self.h2 = 25

    # h3 para o no de saida (10 because pois nos temos 10 digitos diferentes)
    self.h2 = 10

    # learning_rate
    self.learning_rate = 1e-5

  def initialize_weights(self,l0,l1):
    w = np.random.randn(l0,l1)*0.01
    b = np.zeros((1,l1))

    return w,b
    

  def forward_prop(self,X,parameters):
    W2 = parameters['W2']
    W1 = parameters['W1']
    b2 = parameters['b2']
    b1 = parameters['b1']

    #forward prob
    a0 = X
    z1 = np.dot(a0,W1) + b1

    # aplicando a não linearidade relu
    a1 = np.maximum(0,z1)
    z2 = np.dot(a1,W2) + b2

    #softmax na the ultima camada
    scores = z2
    exp_scores = np.exp(scores)
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)


    #valores caches fromward pass to use for backward pass
    cache = {'a0' : X,
             'probs' : probs,
             'a1' : a1
    }

    return cache, probs

  def compute_cost(self,y,probs,parameters):
    W2 = parameters['W2']
    W1 = parameters['W1']

    loss = -np.log(probs[np.arange(self.m),y])
    avg_loss = np.sum(loss) /self.m

    return avg_loss


  def backward_prop(self, cache, parameters, y):

    #descarregamento a partir dos parametros
    W2 = parameters['W2']
    W1 = parameters['W1']
    b2 = parameters['b2']
    b1 = parameters['b1']

    # startar as probabilidades a partir das probabilidades forward
    a0 = cache['a0']
    a1 = cache['a1']
    probs = cache['probs']

    # startar o backpropagation
    dz2 = probs
    dz2[np.arange(self.m),y] -= 1
    dz2 /= self.m

    # backprop a partir dos valores dW2 e db2
    dW2 = np.dot(a1.T, dz2)
    db2 = np.sum(dz2, axis=0, keepdims=True)

    # backprop na ultima camada 
    dz1 = np.dot(dz2, W2.T)
    dz1 = dz1*(a1 > 0)

    # Backpropagation atraves do dW1, db1
    dW1 = np.dot(a0.T, dz1)
    db1 = np.sum(dz1, axis=0, keepdims=True)

    grads = {
        'dW1' : dW1,
        'dW2' : dW2,
        'db1' : db1,
        'db2' : db2
    }

    return grads

  def update_parameters(self,parameters,grads):
    learning_rate=self.learning_rate

    W2 = parameters['W2']
    W1 = parameters['W1']
    b2 = parameters['b2']
    b1 = parameters['b1']

    dW2 = grads['dW2']
    dW1 = grads['dW1']
    db2 = grads['db2']
    db1 = grads['db1']


    # gradient descent
    W2 -= learning_rate*dW2
    W1 -= learning_rate*dW1

    b2 -= learning_rate*db2
    b1 -= learning_rate*db1

    parameters = {
        'W1': W1, 'W2': W2, 'b1':b1 , 'b2':b2
    }

    return parameters
  


  def main(self,X,y,num_iter):
    # inicializar nossos pesos

    W1, b1 = self.initialize_weights(self.n, self.h1)
    W2, b2 = self.initialize_weights(self.h1, self.h2)
    
    # pack parameters into a dictionary
    parameters = {'W1':W1, 'W2':W2, 'b1':b1,'b2':b2}

    # How many gradients descent updates we want to do
    for it in range(num_iter+1):

      # forward prop
      cache, probs = self.forward_prop(X, parameters)

      # calculate cost
      J = self.compute_cost(y, probs, parameters)

      #print cost sometimes
      if it % 100 == 0:
        print(f'Na interação {it} we have a loss of {J}')

      #back propr
      grads = self.backward_prop(cache, parameters, y)

      #update oaraneterss
      parameters = self.update_parameters(parameters, grads)

    
    return parameters





  






In [58]:
NN = NeuralNetwork(X_train, y_train)

trained_parameters = NN.main(X_train, y_train, 5000)

Na interação 0 we have a loss of 3.367016318876067
Na interação 100 we have a loss of 2.4946851030295885
Na interação 200 we have a loss of 2.193626450411557
Na interação 300 we have a loss of 1.9840536358780387
Na interação 400 we have a loss of 1.808634504285898
Na interação 500 we have a loss of 1.6555361762581724
Na interação 600 we have a loss of 1.5205434554770407
Na interação 700 we have a loss of 1.402483461281722
Na interação 800 we have a loss of 1.2997720051064892
Na interação 900 we have a loss of 1.210445758806331
Na interação 1000 we have a loss of 1.1330397776576784
Na interação 1100 we have a loss of 1.065835203551641
Na interação 1200 we have a loss of 1.0072951842963243
Na interação 1300 we have a loss of 0.9558198308163344
Na interação 1400 we have a loss of 0.9104866958516388
Na interação 1500 we have a loss of 0.8703855036420708
Na interação 1600 we have a loss of 0.83446440858331
Na interação 1700 we have a loss of 0.8022246718743403
Na interação 1800 we have a lo